In [ ]:
import numpy as np # linear algebra
import pandas as pd
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
# tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, TimeDistributed, LeakyReLU, Conv1D, BatchNormalization, MaxPooling1D, AveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping

import pickle
import Model_function as func

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
cluster = 7
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\Cluster_{cluster}\\Cluster_{cluster}_regression_minmax0', 'rb') as fp:
    load_list = pickle.load(fp)

In [ ]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_list[0], load_list[1], load_list[2], load_list[3], load_list[4], load_list[5]

In [ ]:
(len(X_train) == len(Y_train)) and (len(X_val) == len(Y_val)) and (len(X_test) == len(Y_test))

In [ ]:
np.array(X_train)

In [ ]:
y_train, y_val, y_test = func.get_dependent_variable(Y_train, Y_val, Y_test)
y_test

In [ ]:
Y_train

In [ ]:
model = Sequential()
#model.add(Conv1D(
            #filters=100, 
            #kernel_size=3, 
            #activation='relu', 
            #input_shape=(20, 43),
            #padding='causal'))
#model.add(BatchNormalization())
model.add(LSTM(
            name='lstm_0',
            units=100,
            return_sequences=True,
            stateful=False,
            activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(LSTM(
            name='lstm_1',
            units=100,
            return_sequences=True,
            stateful=False,
            activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(LSTM(
            name='lstm_2',
            units=100,
            return_sequences=True,
            stateful=False,
            activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(LSTM(
            name='lstm_3',
            units=100,
            return_sequences=False,
            stateful=False,
            activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
#model.add(Dense(512, activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.Nadam(lr=0.001),
                metrics=['accuracy'])
    
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model.fit(np.array(X_train), Y_train_ATR, validation_data=(np.array(X_val), Y_val_ATR), batch_size = 256, epochs=20, verbose=2, callbacks=[early_stopping])
    

In [ ]:
model = Sequential()
#model.add(Conv1D(
            #filters=100, 
            #kernel_size=3, 
            #activation='relu', 
            #input_shape=(20, 43)))
#model.add(BatchNormalization())
model.add(LSTM(
            name='lstm_0',
            units=100,
            return_sequences=True,
            stateful=False,
            activation='relu'))
model.add(LSTM(
            name='lstm_1',
            units=100,
            return_sequences=True,
            stateful=False,
            activation='relu'))
model.add(LSTM(
            name='lstm_2',
            units=100,
            return_sequences=True,
            stateful=False,
            activation='relu'))
model.add(LSTM(
            name='lstm_3',
            units=100,
            return_sequences=False,
            stateful=False,
            activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error',
                optimizer=optimizers.Nadam(lr=0.001),
                metrics=['mape'])
    
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), batch_size = 256, epochs=20, verbose=2, callbacks=[early_stopping])
    

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(np.array(X_train).shape[1], np.array(X_train).shape[2]),padding='causal'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error',
                optimizer=optimizers.Nadam(lr=0.001),
                metrics=['mape'])


model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), batch_size = 256, epochs=200, verbose=2)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(20, 43), padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.Nadam(lr=0.001),
                metrics=['accuracy'])
    
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model.fit(np.array(X_train), Y_train_ATR, validation_data=(np.array(X_val), Y_val_ATR), batch_size = 256, epochs=80, verbose=2)

In [ ]:
score = model.evaluate(np.array(X_test), np.array(y_test), batch_size=4096)

In [ ]:
score

In [ ]:
first_layer_weights = model.layers[0].get_weights()[0]
second_layer_weights = model.layers[2].get_weights()[0]
third_layer_weights = model.layers[4].get_weights()[0]
forth_layer_weights = model.layers[7].get_weights()[0]

In [ ]:
model.layers

In [ ]:
first_layer_weights[1].mean(axis=1)

In [ ]:
prediction = model.predict(np.array(X_test),batch_size=4096)

In [ ]:
pred = func.transform_back(prediction, 7, scaler='minmax_zero')
pred

In [ ]:
d = func.get_prediction_label(pred, Y_test, 7, problem='Weekly')

In [ ]:
df_list = [group[1] for group in d.groupby(d['StockNo'])]
s = []
for df in df_list:
    s.append([func.Regression_Evaluation(df, problem='Weekly'), df['StockName'].iloc[0]])
max(s)

In [ ]:
pred.tolist()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import date
df_list = [group[1] for group in d.groupby(d['StockNo'])]
for df in df_list:
    df = df.reset_index(drop=True)
    plt.figure(figsize=(30,10))
    plt.plot(df['close'])
    plt.plot(df['prediction'], linewidth=0.5)
    plt.title(df['StockNo'].iloc[0])

In [ ]:
d = d.reset_index(drop=True)

In [ ]:
func.Regression_Evaluation(d, problem='Weekly')

In [ ]:
j = 0
for item in s:
    if item[0] > 0.5:
        j += 1
    else:
        continue
        
j

In [ ]:
d[['']]

In [ ]:
d[['ts','label_weekly_ATR', 'label_pred', 'close', 'prediction', 'WeeklyReturn', 'ATR', 'return_pred']]

In [ ]:
model.save('CNN.h5')

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model('CNN.h5')

In [ ]:
prediction

In [ ]:
new_model.summary()

In [ ]:
prediction1

In [ ]:
r = prediction.tolist()
y_pred = []
for i, item in tqdm(enumerate(r), total=len(r)):
    y_pred.append(item.index(max(item)))

    
y_true = [item.tolist().index(1) for item in Y_test_ATR]

In [ ]:
y_pred.count(1)/len(y_pred)

In [ ]:
y_true.count(1)/len(y_true)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))